# Part 2 — Finding duplicates and near duplicates using WebSty

The goal of this tutorial is to use WebSty to detect groups of similar documents. Then for each group select the most representative document that will represent given group.

### Actions:
1. Upload sample dataset to CLARIN-PL WS
2. Run the WebSty task on the uploaded dataset
3. Download the similarity matrix
4. Create groups of similar documents
5. Analyze the similarity of document pairs

### Links and resources

1. http://ws.clarin-pl.eu/websty.shtml
2. http://clarin-pl.eu/wp-content/uploads/2017/02/WebSty.piatek.I.1805.pdf
3. https://www.dropbox.com/s/54gmpdd6x3rx4gq/brexit_pl.zip?dl=1

## 1. Uploading the dataset

In [1]:
import json
import requests

clarinpl_url = "http://ws.clarin-pl.eu/nlprest2/base"
user_mail = "demo2019@nlpday.pl"

In [2]:
import urllib.request

url = clarinpl_url + "/upload/"
url_zip = "https://www.dropbox.com/s/54gmpdd6x3rx4gq/brexit_pl.zip?dl=1"

doc = urllib.request.urlopen(url_zip).read()

docuemnts_zip = "documents.zip"
file = open(docuemnts_zip, "w+b")
file.write(doc)
file.close()
print("Dataset saved to %s" % docuemnts_zip)
    
print("Size of the package: %d" % len(doc))

Dataset saved to documents.zip
Size of the package: 800523


In [3]:
headers = {'content-type': 'binary/octet-stream'}

file_handler = requests.post(url, data=doc, headers=headers).text
print("File handler: %s" % file_handler)
print("URL: %s/download%s" % (clarinpl_url, file_handler))

File handler: /users/default/b8a4afe1-9322-4867-bf23-a18aad0527af
URL: http://ws.clarin-pl.eu/nlprest2/base/download/users/default/b8a4afe1-9322-4867-bf23-a18aad0527af


## 2. Run the WebSty task 

In [4]:
import time

url = clarinpl_url + "/startTask"

lpmn = 'filezip(%s)|div(20000)|wcrft2' % file_handler
lpmn += '|fextor2({"features":"base interp_signs bigrams","base_modification":"startlist","orth_modification":"startlist","lang":"pl","filters":{"base":[{"type":"lemma_stoplist","args":{"stoplist":"@resources/fextor/ml/polish_base_startlist.txt"}}]}})'
lpmn += '|dir|out("output_fextor")'
lpmn += '|featfilt2({"weighting":"all:sm-mi_simple","filter":"min_tf-1 min_df-1","similarity":"cosine"})'
lpmn += '|cluto({"no_clusters":10,"analysis_type":"plottree"})'

print("LPMN: %s" % lpmn)

payload = {'lpmn': lpmn, 'user': user_mail}
headers = {'content-type': 'application/json'}

start = time.time()
task_id = requests.post(url, data=json.dumps(payload), headers=headers).text
print("Task id: %s" % task_id)

# Check task status
processing = True
file_id = None

while processing:
  data = requests.get(clarinpl_url + "/getStatus/" + task_id).text
  result = json.loads(data)
  end = time.time()
  if result["status"] == "PROCESSING":
    print("[%3d s] Status: %s; Progress: %6.2f%%" % (end-start, result["status"], result["value"]*100))
    time.sleep(1)
  elif result["status"] == "DONE":
    print(result)
    file_id = result["value"]['result'][0]["fileID"]
    processing = False  
    print("[%3d s] Status: DONE      ; Progress: 100.00%%" % (end-start))
  else:
    print(data)
    processing = False  
    
print("Result file id: %s" % file_id)

LPMN: filezip(/users/default/b8a4afe1-9322-4867-bf23-a18aad0527af)|div(20000)|wcrft2|fextor2({"features":"base interp_signs bigrams","base_modification":"startlist","orth_modification":"startlist","lang":"pl","filters":{"base":[{"type":"lemma_stoplist","args":{"stoplist":"@resources/fextor/ml/polish_base_startlist.txt"}}]}})|dir|out("output_fextor")|featfilt2({"weighting":"all:sm-mi_simple","filter":"min_tf-1 min_df-1","similarity":"cosine"})|cluto({"no_clusters":10,"analysis_type":"plottree"})
Task id: dbe82a61-f4a9-4da1-8857-09f6e2f93f66
[  0 s] Status: PROCESSING; Progress:   0.00%
[  1 s] Status: PROCESSING; Progress:   0.00%
[  2 s] Status: PROCESSING; Progress:  13.12%
[  4 s] Status: PROCESSING; Progress:  27.03%
[  5 s] Status: PROCESSING; Progress:  50.67%
[  6 s] Status: PROCESSING; Progress:  65.31%
[  7 s] Status: PROCESSING; Progress:  80.16%
[  8 s] Status: PROCESSING; Progress:  99.53%
[ 10 s] Status: PROCESSING; Progress:  99.53%
[ 11 s] Status: PROCESSING; Progress:  9

## 3. Download the similarity matrix

In [5]:
# clusters.json  clutoout.txt  data.json  distance.json  labels.json  
# matrix.txt  result.clustering  result.json  result.png  result.xlsx  
# rowlabels.pkl  similarity.json  weighted.json

# import numpy as np

url = clarinpl_url + "/download" + file_id + "/similarity.json"
print(url)
data = requests.get(url).content.decode("utf-8-sig")
print("Data size: %d" % len(data))

http://ws.clarin-pl.eu/nlprest2/base/download/requests/cluto/eddc4510-d0cb-4088-9b1a-58ac1d71469d/similarity.json
Data size: 1318748


In [6]:
parsed = json.loads(data)

rowlabels = parsed["rowlabels"]
similarities = parsed["arr"]

document_similarity = []
for x in range(0,len(rowlabels)):
    for y in range(0,x):
        sim = similarities[x][y]
        if sim > 0.0:
            document_similarity.append((sim, x, y))
            
document_similarity_sorted = sorted(document_similarity, key=lambda t: t[0], reverse=True)
            
for t in document_similarity_sorted[:20]:
    print("%6.4f %5d %5d" % t)

0.9992   434   286
0.9928   388   234
0.9913   435   432
0.9905   195   151
0.9902   405   138
0.9890   377    27
0.9872   456   253
0.9843   422    98
0.9803   434   377
0.9795   377   286
0.9794   286    27
0.9790   445   405
0.9777   434    27
0.9775   432   251
0.9754   435   251
0.9752   397   134
0.9747   313    31
0.9731   363    18
0.9696   445   138
0.9585   401   380


## 4. Create groups of similar documents

Initial similarity matrix
```
1.0  0.8  0.7  0.0  0.0
0.8  1.0  0.8  0.0  0.0
0.7  0.8  1.0  0.3  0.2
0.0  0.0  0.3  1.0  0.9
0.0  0.0  0.2  0.9  1.0
```

Filtering similar documents with a threshold (0.7)
```
1.0  1.0  1.0  ---  ---
1.0  1.0  1.0  ---  ---
1.0  1.0  1.0  ---  ---
---  ---  ---  1.0  1.0
---  ---  ---  1.0  1.0
```


In [7]:
import numpy as np

def create_neighbourhood_matrix(similarities, threshold):
    matrix = np.zeros(shape=(len(similarities), len(similarities)))
    for x in range(0,len(rowlabels)):
        for y in range(0,len(rowlabels)):
            matrix[x,y] = 1 if similarities[x][y] >= threshold else 0
    return matrix

matrix = create_neighbourhood_matrix(similarities, 0.7)

In [8]:
def get_row_index_with_highest_sum(matrix):
    sums = [np.sum(matrix[i,:]) for i in range(0, len(matrix))]
    return sums.index(max(sums))
 
def add_into_group(matrix, ind):
    change = True
    indexes = []
    for col in range(len(matrix)):
        if matrix[ind, col] == 1:
            indexes.append(col)
    while change == True:
        change = False
        numIndexes = len(indexes)
        for i in indexes:
            for col in range(len(matrix)):
                if matrix[i, col] == 1:
                    if col not in indexes:
                        indexes.append(col)
        numIndexes2 = len(indexes)
        if numIndexes != numIndexes2:
            change = True
    return indexes
 
def reset_rows_and_cols(matrix, indexes):
    for i in indexes:
        matrix[i,:] = 0
        matrix[:,i] = 0
    return matrix

def cluster_matrix(matrix):
    groups = []
    while np.sum(matrix) > 0:
        group = []
        row = get_row_index_with_highest_sum(matrix)
        indexes = add_into_group(matrix, row)
        groups.append(indexes)
        matrix = reset_rows_and_cols(matrix, indexes)
    return groups

groups = cluster_matrix(matrix)

In [9]:
print ("Number of distinct groups: %d" % len(groups))

print ("Groups with more than one element:")
for i in range(len(groups)):
    group = groups[i]
    if len(group) > 1:
        print("  {}) {}".format(i+1, sorted(group)))

Number of distinct groups: 432
Groups with more than one element:
  1) [36, 62, 76, 119, 160]
  2) [49, 65, 72, 267, 420]
  3) [130, 344, 361, 362, 406]
  4) [27, 286, 377, 434]
  5) [40, 71, 319, 484]
  6) [138, 405, 423, 445]
  7) [251, 274, 432, 435]
  8) [30, 93, 427]
  9) [140, 340, 357]
  10) [0, 57]
  11) [3, 260]
  12) [4, 273]
  13) [6, 23]
  14) [18, 363]
  15) [19, 342]
  16) [26, 297]
  17) [31, 313]
  18) [44, 165]
  19) [47, 81]
  20) [92, 486]
  21) [98, 422]
  22) [106, 374]
  23) [111, 389]
  24) [126, 348]
  25) [134, 397]
  26) [139, 376]
  27) [142, 464]
  28) [151, 195]
  29) [167, 255]
  30) [179, 480]
  31) [182, 328]
  32) [184, 240]
  33) [196, 215]
  34) [202, 421]
  35) [205, 327]
  36) [224, 229]
  37) [234, 388]
  38) [242, 446]
  39) [248, 429]
  40) [253, 456]
  41) [276, 347]
  42) [320, 471]
  43) [338, 402]
  44) [356, 444]
  45) [358, 457]
  46) [367, 490]
  47) [380, 401]
  48) [383, 467]
  49) [394, 491]


## 5. Comparing documents within the groups

Compare documents from 10th group.

In [10]:
import zipfile

doc_a = rowlabels[0]
doc_b = rowlabels[57]

def restore_zip_filename(filename):
    return filename.replace("%", "/")[:-2]

doc_a = restore_zip_filename(doc_a)
doc_b = restore_zip_filename(doc_b)

zf = zipfile.ZipFile(docuemnts_zip, 'r')
doc_a_content = zf.read(doc_a).decode("utf-8-sig")
doc_b_content = zf.read(doc_b).decode("utf-8-sig")

print(doc_a_content)
print(doc_b_content)

pl-465
pl
2018-01-17
http://www.gazetaprawna.pl/artykuly/1098490,izba-gmin-przeglosowala-ustawe-o-wyjsciu-z-ue.html
Wielka Brytania: Izba Gmin przegłosowała ustawę o wyjściu z UE

Uchwalona Ustawa o Wyjściu z Unii Europejskiej odwołuje akt z 1972 roku, na mocy którego Wielka Brytania stała się członkiem Wspólnoty. "Ta ustawa jest kluczowa dla przygotowania kraju do historycznego kroku, jakim będzie wystąpienie z Unii Europejskiej" - powiedział jeszcze przed głosowaniem minister ds. Brexitu David Davis. Wynik głosowania stanowi sukces premier Theresy May nad oponentami politycznymi, którzy życzyliby sobie bardziej "miękkiego" rozwodu z UE , ale w Izbie Lordów, gdzie Partia Konserwatywna nie ma większości, może nie być tak łatwo - komentuje agencja Reutera.

Wyższa izba parlamentu zacznie teraz analizę ustawy , co może trwać kilka miesięcy - zaznacza agencja. Wszelkie zmiany w tekście będą musiały być zaaprobowane przez Izbę Gmin. Oznacza to, że cały proces legislacyjny może potrwać do m

### Compare documents using difflib

In [11]:
from termcolor import colored
import difflib

class Word:

    def __init__(self, orth, after):
        self.orth = orth
        self.after = after

        
class DocumentDiff:
    
    def __init__(self, s1, s2):
        self.s1 = s1
        self.s2 = s2
        self.overlap = 0.0
        self.diff1 = ""
        self.diff2 = ""
        self.calculate()
        
    def seq(self, index, words, begin, end, color):
        str = ""
        for w in words[begin:end]:
            str += w.orth + w.after
        return colored("[%d]" % index, 'grey') + colored(str, color)
                
    def calculate(self):
        w1 = [w.orth for w in self.s1]
        w2 = [w.orth for w in self.s2]
        matcher = difflib.SequenceMatcher(None, w1, w2)
        index = 1
        for tag, i1, i2, j1, j2 in matcher.get_opcodes():   
            if tag == 'delete':
                self.diff1 += self.seq(index, self.s1, i1, i2, 'green')
            elif tag == 'equal':
                self.overlap += (i2-i1)
                self.diff1 += self.seq(index, self.s1, i1, i2, 'red')
                self.diff2 += self.seq(index, self.s2, j1, j2, 'red')                
            elif tag == 'insert':
                self.diff2 += self.seq(index, self.s2, j1, j2, 'green')                
            elif tag == 'replace':
                self.diff1 += self.seq(index, self.s1, i1, i2, 'blue')
                self.diff2 += self.seq(index, self.s2, j1, j2, 'blue')                
            index += 1
            
    def get_overlap_doc1(self):
        return self.overlap/len(self.s1)
    
    def get_overlap_doc2(self):
        return self.overlap/len(self.s2)
            
    def print_diff(self):
        print("-"*100)
        print(self.diff1)
        print("Overlap ratio: %6.4f" % self.get_overlap_doc1())
        print("-"*100)
        print(self.diff2)
        print("Overlap ratio: %6.4f" % self.get_overlap_doc2())
        print("-"*100)
        
        
s1 = [Word(w, " ") for w in "Ala ma kota i psa".split(" ")]
s2 = [Word(w, " ") for w in "Ania ma czarnego kota".split(" ")]

DocumentDiff(s1, s2).print_diff()

print("Colors: " + colored("in both", 'red') + ", "\
      + colored("replaced", 'blue') + ", "\
      + colored("not present in the other text", 'green'))

----------------------------------------------------------------------------------------------------
[1]Ala [2]ma [4]kota [5]i psa 
Overlap ratio: 0.4000
----------------------------------------------------------------------------------------------------
[1]Ania [2]ma [3]czarnego [4]kota 
Overlap ratio: 0.5000
----------------------------------------------------------------------------------------------------
Colors: in both, replaced, not present in the other text


### Document as an array of text blocks splitted by new line character

In [12]:
s1 = [Word(w, "\n") for w in doc_a_content.split("\n")]
s2 = [Word(w, "\n") for w in doc_b_content.split("\n")]

DocumentDiff(s1, s2).print_diff()

----------------------------------------------------------------------------------------------------
[1]pl-465
[2]pl
[3]2018-01-17
http://www.gazetaprawna.pl/artykuly/1098490,izba-gmin-przeglosowala-ustawe-o-wyjsciu-z-ue.html
[4]Wielka Brytania: Izba Gmin przegłosowała ustawę o wyjściu z UE

[5]Uchwalona Ustawa o Wyjściu z Unii Europejskiej odwołuje akt z 1972 roku, na mocy którego Wielka Brytania stała się członkiem Wspólnoty. "Ta ustawa jest kluczowa dla przygotowania kraju do historycznego kroku, jakim będzie wystąpienie z Unii Europejskiej" - powiedział jeszcze przed głosowaniem minister ds. Brexitu David Davis. Wynik głosowania stanowi sukces premier Theresy May nad oponentami politycznymi, którzy życzyliby sobie bardziej "miękkiego" rozwodu z UE , ale w Izbie Lordów, gdzie Partia Konserwatywna nie ma większości, może nie być tak łatwo - komentuje agencja Reutera.
[6]
[7]Wyższa izba parlamentu zacznie teraz analizę ustawy , co może trwać kilka miesięcy - zaznacza agencja. Wszelkie

### Document as an array of words splitted by single spaces

In [13]:
s1 = [Word(w, " ") for w in doc_a_content.split(" ")]
s2 = [Word(w, " ") for w in doc_b_content.split(" ")]

DocumentDiff(s1, s2).print_diff()

----------------------------------------------------------------------------------------------------
[1]pl-465
pl
2018-01-17
http://www.gazetaprawna.pl/artykuly/1098490,izba-gmin-przeglosowala-ustawe-o-wyjsciu-z-ue.html
Wielka [2]Brytania: Izba Gmin przegłosowała ustawę o wyjściu z UE

Uchwalona Ustawa o Wyjściu z Unii Europejskiej odwołuje akt z 1972 roku, na mocy którego Wielka Brytania stała się członkiem Wspólnoty. [3]"Ta [4]ustawa jest kluczowa dla przygotowania kraju do historycznego kroku, jakim będzie wystąpienie z Unii [5]Europejskiej" - [6]powiedział jeszcze przed głosowaniem minister ds. Brexitu David Davis. Wynik głosowania stanowi sukces premier Theresy May nad oponentami politycznymi, którzy życzyliby sobie bardziej [7]"miękkiego" [8]rozwodu z [9]UE , [10]ale w Izbie Lordów, gdzie Partia Konserwatywna nie ma większości, może nie być tak łatwo [11]- [12]komentuje agencja [13]Reutera.

Wyższa [14]izba parlamentu zacznie teraz analizę [15]ustawy , [16]co może trwać kilka mie

### Document as an array of token orths

#### Auxiliary class for tokenization

In [14]:
import xml.etree.ElementTree as ET

class Tokenizer:
    
    def __init__(self):
        self.url = "http://ws.clarin-pl.eu/nlprest2/base/process"
        self.user_mail = "demo2019@nlpday.pl"
        self.lpmn = "wcrft2"
        
    def process(self, text):
        payload = {'text': text, 'lpmn': self.lpmn, 'user': self.user_mail}
        headers = {'content-type': 'application/json'}
        r = requests.post(self.url, data=json.dumps(payload), headers=headers)
        return r.content.decode('utf-8')        
    
    def orths(self, text):
        ccl = self.process(text)
        tree = ET.fromstring(ccl)
        return [orth.text for orth in tree.iter('orth')]

#### Convert document into an array of orth tokens

In [15]:
tokenizer = Tokenizer()

s1 = [Word(w, " ") for w in tokenizer.orths(doc_a_content)]
s2 = [Word(w, " ") for w in tokenizer.orths(doc_b_content)]
    
DocumentDiff(s1, s2).print_diff()

----------------------------------------------------------------------------------------------------
[1]pl - [2]465 [3]pl [4]2018-01-17 http://www.gazetaprawna.pl/artykuly/1098490,izba-gmin-przeglosowala-ustawe-o-wyjsciu-z-ue.html [5]Wielka Brytania : Izba Gmin przegłosowała ustawę o wyjściu z UE Uchwalona Ustawa o Wyjściu z Unii Europejskiej odwołuje akt z 1972 roku , na mocy którego Wielka Brytania stała się członkiem Wspólnoty . [6]" [7]Ta ustawa jest kluczowa dla przygotowania kraju do historycznego kroku , jakim będzie wystąpienie z Unii Europejskiej [8]" - [9]powiedział jeszcze przed głosowaniem minister ds . Brexitu David Davis . Wynik głosowania stanowi sukces premier Theresy May nad oponentami politycznymi , którzy życzyli by sobie bardziej [10]" [11]miękkiego [12]" [13]rozwodu z UE , ale w Izbie Lordów , gdzie Partia Konserwatywna nie ma większości , może nie być tak łatwo [14]- [15]komentuje agencja Reutera . Wyższa izba parlamentu zacznie teraz analizę ustawy , co może trwa

In [16]:
def align_tokens_with_text(tokens, text):
    n = 0
    words = []
    last_orth = None    
    for orth in tokens:
        m = text.find(orth.orth, n)
        if last_orth != None:
            words += [Word(last_orth, text[n:m])]
        last_orth = orth.orth
        n = m + len(orth.orth)
    words += [Word(last_orth, "")]
    return words

s1_aligned = align_tokens_with_text(s1, doc_a_content)
s2_aligned = align_tokens_with_text(s2, doc_b_content)

DocumentDiff(s1_aligned, s2_aligned).print_diff()

----------------------------------------------------------------------------------------------------
[1]pl-[2]465
[3]pl
[4]2018-01-17
http://www.gazetaprawna.pl/artykuly/1098490,izba-gmin-przeglosowala-ustawe-o-wyjsciu-z-ue.html
[5]Wielka Brytania: Izba Gmin przegłosowała ustawę o wyjściu z UE

Uchwalona Ustawa o Wyjściu z Unii Europejskiej odwołuje akt z 1972 roku, na mocy którego Wielka Brytania stała się członkiem Wspólnoty. [6]"[7]Ta ustawa jest kluczowa dla przygotowania kraju do historycznego kroku, jakim będzie wystąpienie z Unii Europejskiej[8]" - [9]powiedział jeszcze przed głosowaniem minister ds. Brexitu David Davis. Wynik głosowania stanowi sukces premier Theresy May nad oponentami politycznymi, którzy życzyliby sobie bardziej [10]"[11]miękkiego[12]" [13]rozwodu z UE , ale w Izbie Lordów, gdzie Partia Konserwatywna nie ma większości, może nie być tak łatwo [14]- [15]komentuje agencja Reutera.

Wyższa izba parlamentu zacznie teraz analizę ustawy , co może trwać kilka miesięc

### Analysis of a group with more than two documents

In [17]:
group = groups[0]
print(group)

[36, 62, 76, 119, 160]


In [18]:
document_tokens = []

for i in group:
    print("Document index: %d" % i)
    doc = rowlabels[i]
    doc = restore_zip_filename(doc)
    doc_content = zf.read(doc).decode("utf-8-sig")
    
    tokens = [Word(w, " ") for w in tokenizer.orths(doc_content)]
    tokens_aligned = align_tokens_with_text(tokens, doc_content)
    document_tokens.append(tokens_aligned)
    
print("Ready")

Document index: 36
Document index: 62
Document index: 76
Document index: 119
Document index: 160
Ready


In [19]:
overlap_matrix = np.zeros(shape=(len(document_tokens), len(document_tokens)))

for i in range(0, len(group)):    
    for j in range(0, i):
        dd = DocumentDiff(document_tokens[i], document_tokens[j])
        overlap_matrix[i,j] = dd.get_overlap_doc1()
        overlap_matrix[j,i] = dd.get_overlap_doc2()
        
        
print("Document overlap")
print(overlap_matrix)
print("")

for i in range(0, len(document_tokens)):
    row = overlap_matrix[i,]
    row = np.delete(row, i)
    avg = sum(row)/len(row)
    print("[%d] %6.3f" % (i, avg))

Document overlap
[[0.         0.95495495 0.93468468 0.83558559 0.85135135]
 [0.86004057 0.         0.85598377 0.75862069 0.75862069]
 [0.87368421 0.88842105 0.         0.80210526 0.77684211]
 [0.93450882 0.94206549 0.95969773 0.         0.82871537]
 [0.81641469 0.80777538 0.79697624 0.71058315 0.        ]]

[0]  0.894
[1]  0.808
[2]  0.835
[3]  0.916
[4]  0.783


[Back to agenda](agenda.ipynb)